### STEP 1: Installing required libraries

In [ ]:
!pip install spacy rapidfuzz pandas openpyxl
!python -m spacy download en_core_web_sm

import spacy
from rapidfuzz import process, fuzz
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


###STEP 2: Load spaCy model

In [ ]:
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


###STEP 3: Load KPI Dataset

In [ ]:
#Load KPI Dataset
from google.colab import files
uploaded = files.upload()

Saving test_data_modified_3.xlsx to test_data_modified_3 (1).xlsx


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import wordnet
from collections import defaultdict
from difflib import SequenceMatcher
from itertools import permutations

# Download necessary NLTK resources
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download("omw-1.4")
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Load KPI Dataset (assuming it's uploaded to Colab)
from google.colab import files
uploaded = files.upload()

# Load the dataset
file_path = "test_data_modified_3.xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Extract unique KPI names
kpi_list = df["KPI"].dropna().unique().tolist()

# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to tokenize and clean text
def tokenize(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    return [lemmatizer.lemmatize(word) for word in word_tokenize(text)]  # Lemmatize tokens

# Function to get synonyms using WordNet
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))  # Replace underscores with spaces
    return synonyms

# Function to compute Jaccard similarity
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

# Function to compute Levenshtein similarity
def levenshtein_similarity(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

# Function to find best matching KPI
def find_best_kpi(query):
    query_tokens = set(tokenize(query))

    # Extract important words (nouns/adjectives) for better context matching
    pos_tags = nltk.pos_tag(query_tokens)
    important_words = {word for word, tag in pos_tags if tag in ["NN", "NNS", "JJ"]}  # Nouns & adjectives
    expanded_query = query_tokens.copy()

    # Expand query with synonyms
    for word in important_words:
        expanded_query.update(get_synonyms(word))

    similarity_scores = []

    for kpi in kpi_list:
        kpi_tokens = set(tokenize(kpi))

        # Compute Jaccard and Levenshtein similarities
        jaccard_sim = jaccard_similarity(expanded_query, kpi_tokens)
        levenshtein_sim = levenshtein_similarity(query, kpi)

        # **Boost KPI relevance if it contains numeric patterns (like %)**
        num_weight = 1.2 if "%" in kpi else 1.0

        # **Increase weight for exact word matches**
        exact_match_bonus = 0.3 if len(query_tokens & kpi_tokens) > 0 else 0

        # Combine scores with weighted importance
        total_score = (0.5 * jaccard_sim) + (0.4 * levenshtein_sim) + (exact_match_bonus)
        total_score *= num_weight  # Apply weight adjustment

        similarity_scores.append((kpi, total_score))

    # Sort by highest similarity score
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    return similarity_scores[:5]  # Return top 5 matches

# Example queries
query = "Which group


###STEP 4: Extract KPI List

In [ ]:
#Extract KPI List
df = pd.read_excel("test_data_modified_3.xlsx")
kpi_list = df["KPI"].dropna().unique().tolist()

In [ ]:
kpi_list

['Certified Employee in Current GC',
 'Manpower Tracked for Certification- Competency',
 'Loss Time Injuries',
 'Loss Time Injuries (Contractor)',
 'Off-Job Man Hours (Emp)',
 'On-Job Man Hours (Contractor)',
 'On-Job Man Hours (EMP + Cont)',
 'On-Job Man Hours (Emp)',
 'Positive Responses to the Phishing Tests',
 'APNE/VCGNE',
 'AV & Office Devices',
 'App re-opened maint incidents',
 'Application Cost/Per Hr. (In-House)',
 'Applications Investment Capacity',
 'Applications Maintain Capacity',
 'Applications Run Capacity',
 'Assets Net Book Value',
 'Attrition',
 'Average Company Workforce',
 'BI-19 No. Overdue Projects (Past ERC)',
 'BI-19 No. of Projects with ? 3 Months To ERC',
 'Biz Travel',
 'Business Sustainability - Below Target',
 'Business Sustainability - Meeting Target',
 'Business Sustainability - Yellow',
 'CDPNE',
 'CPH Positions - Total',
 'CPH Positions Status - Acting',
 'CPH Positions Status - Permanent',
 'CPH Positions Status - Vacant',
 'CRM Delivery – Number of S

###STEP 5: Extract Potential KPI Entities:
  <ol>1. Remove Noise words</ol>
  <ol>2. Extract entities that are relevant to the query</ol>
  <ol>3. Extract entities using bigrams and trigrams for better context understanding</ol>
  <ol>4. Combine KPI entities extracted earlier</ol>



In [ ]:
def extract_kpi_entities(query):
    doc = nlp(query)

    # Exclude common words that create noise
    exclude_words = {"plan", "beginning", "cost", "total", "overall"}

    words = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN", "ADJ","ORG","MONEY","PERCENT","QUANTITY"] and token.text.lower() not in exclude_words]
    bigrams = [" ".join(words[i:i+2]) for i in range(len(words)-1)]
    trigrams = [" ".join(words[i:i+3]) for i in range(len(words)-2)]

    extracted_entities = words + bigrams + trigrams
    print(f"Extracted Entities (Final Version): {extracted_entities}")

    return extracted_entities if extracted_entities else [query]

###STEP 6: Extract Top Matching KPIs
<ol>1. Ignore generic words unless part of a phrase </ol>
<ol>2. Adjust threshold for shorter terms </ol>
<ol>3. Apply fuzzy matching with adjusted threshold </ol>
<ol>4. Exact substring fallback for acronyms and dates</ol>
<ol>5. Handle "Number of X" cases (ensure relevant filtering)</ol>
<ol>6. Prioritize longer matches</ol>

In [ ]:
def get_top_matching_kpis(query, top_n=3):
    extracted_entities = extract_kpi_entities(query)
    matched_kpis = []

    for entity in extracted_entities:
        if entity.lower() in {"group", "highest", "number", "plan", "cost", "total"}:
            continue

        # Lowered thresholds
        threshold = 65 if len(entity) <= 4 else 75

        best_matches = process.extract(entity, kpi_list, scorer=fuzz.token_set_ratio, limit=top_n)

        # Debug print to check potential matches before filtering
        print(f"Potential Matches for '{entity}': {best_matches}")

        filtered_matches = [match[0] for match in best_matches if match[1] >= threshold]

        if not filtered_matches:
            filtered_matches = [kpi for kpi in kpi_list if entity.lower() in kpi.lower()]

        if "number" in entity.lower():
            filtered_matches = [kpi for kpi in filtered_matches if any(word in kpi.lower() for word in extracted_entities)]

        print(f"Filtered Matches for '{entity}': {filtered_matches}")
        matched_kpis.extend(filtered_matches)

    matched_kpis = sorted(set(matched_kpis), key=len, reverse=True)
    return matched_kpis

###STEP 7: Use Relevant Queries to extract KPIs

In [ ]:
# Test query
query = "Which group in CAD has the highest % of female representation?"
print(get_top_matching_kpis(query))


Extracted Entities (Final Version): ['group', 'CAD', 'highest', '%', 'female', 'representation', 'group CAD', 'CAD highest', 'highest %', '% female', 'female representation', 'group CAD highest', 'CAD highest %', 'highest % female', '% female representation']
Potential Matches for 'CAD': [('CDPNE', 50.0, 25), ('MVA', 33.33333333333333, 97), ('IKTVA', 25.0, 80)]
Filtered Matches for 'CAD': []
Potential Matches for '%': [('Certified Employee in Current GC', 0.0, 0), ('Manpower Tracked for Certification- Competency', 0.0, 1), ('Loss Time Injuries', 0.0, 2)]
Filtered Matches for '%': []
Potential Matches for 'female': [('Female CPH', 62.5, 72), ('Female Representation', 37.03703703703704, 73), ('Total Spend', 35.294117647058826, 368)]
Filtered Matches for 'female': ['Female CPH', 'Female Representation', 'Permanent Female Employees', 'Permanent CPH Female Employees']
Potential Matches for 'representation': [('Female Representation', 74.28571428571428, 73), ('Services Utilization', 58.82352

In [ ]:
get_top_matching_kpis("How many patents were granted for SCMD in April 2024?")

Extracted Entities (Final Version): ['many', 'patents', 'SCMD', 'April', 'many patents', 'patents SCMD', 'SCMD April', 'many patents SCMD', 'patents SCMD April']
Potential Matches for 'many': [('Consultancy', 40.0, 50), ('Compliant CRM', 35.294117647058826, 46), ('Communications', 33.33333333333333, 44)]
Filtered Matches for 'many': []
Potential Matches for 'patents': [('Patents Filed', 60.0, 149), ('Patents Granted', 54.54545454545455, 150), ('Compliant Incident', 48.0, 47)]
Filtered Matches for 'patents': ['Patents Filed', 'Patents Granted']
Potential Matches for 'SCMD': [('SMPs', 50.0, 172), ('CDPNE', 44.44444444444444, 25), ('Closed CRM', 28.57142857142857, 36)]
Filtered Matches for 'SCMD': []
Potential Matches for 'April': [('Attrition', 42.857142857142854, 17), ('Total Attrition', 40.0, 185), ('Hiring', 36.36363636363637, 79)]
Filtered Matches for 'April': []
Potential Matches for 'many patents': [('Patents Granted', 66.66666666666666, 150), ('Patents Filed', 56.0, 149), ('Compli

['Patents Granted', 'Patents Filed']

In [ ]:
get_top_matching_kpis("What was CAD Controllable Cost plan in the beginning of 2024?")

Extracted Entities (Final Version): ['CAD', 'Controllable', 'CAD Controllable']
Potential Matches for 'CAD': [('CDPNE', 50.0, 25), ('MVA', 33.33333333333333, 97), ('IKTVA', 25.0, 80)]
Filtered Matches for 'CAD': []
Potential Matches for 'Controllable': [('Controllable Cost', 100.0, 52), ('Controllable Cost ($)', 100.0, 53), ('Direct Controllable Cost', 100.0, 59)]
Filtered Matches for 'Controllable': ['Controllable Cost', 'Controllable Cost ($)', 'Direct Controllable Cost']
Potential Matches for 'CAD Controllable': [('Controllable Cost', 85.71428571428571, 52), ('Controllable Cost ($)', 85.71428571428571, 53), ('Direct Controllable Cost', 85.71428571428571, 59)]
Filtered Matches for 'CAD Controllable': ['Controllable Cost', 'Controllable Cost ($)', 'Direct Controllable Cost']


['Direct Controllable Cost', 'Controllable Cost ($)', 'Controllable Cost']

In [ ]:
get_top_matching_kpis("What is the Professional Certification % in F&AD in July 2024?")

Extracted Entities (Final Version): ['Professional', 'Certification', '%', 'F&AD', 'July', 'Professional Certification', 'Certification %', '% F&AD', 'F&AD July', 'Professional Certification %', 'Certification % F&AD', '% F&AD July']
Potential Matches for 'Professional': [('Professional Certification', 100.0, 160), ('CPH Positions - Total', 48.484848484848484, 26), ('Total Attrition', 44.44444444444444, 185)]
Filtered Matches for 'Professional': ['Professional Certification']
Potential Matches for 'Certification': [('Professional Certification', 100.0, 160), ('Services Utilization', 60.60606060606061, 353), ('IT Customer Satisfaction', 59.45945945945946, 82)]
Filtered Matches for 'Certification': ['Professional Certification']
Potential Matches for '%': [('Certified Employee in Current GC', 0.0, 0), ('Manpower Tracked for Certification- Competency', 0.0, 1), ('Loss Time Injuries', 0.0, 2)]
Filtered Matches for '%': []
Potential Matches for 'F&AD': [('MVA', 28.57142857142857, 97), ('AV 

['Professional Certification']

In [ ]:
get_top_matching_kpis("Which group in CAD has the highest number of female employees?")

Extracted Entities (Final Version): ['group', 'CAD', 'highest', 'number', 'female', 'employees', 'group CAD', 'CAD highest', 'highest number', 'number female', 'female employees', 'group CAD highest', 'CAD highest number', 'highest number female', 'number female employees']
Potential Matches for 'CAD': [('CDPNE', 50.0, 25), ('MVA', 33.33333333333333, 97), ('IKTVA', 25.0, 80)]
Filtered Matches for 'CAD': []
Potential Matches for 'female': [('Female CPH', 62.5, 72), ('Female Representation', 37.03703703703704, 73), ('Total Spend', 35.294117647058826, 368)]
Filtered Matches for 'female': ['Female CPH', 'Female Representation', 'Permanent Female Employees', 'Permanent CPH Female Employees']
Potential Matches for 'employees': [('SC11-17 employees', 100.0, 169), ('SC11-17 employees possessing a certification', 100.0, 170), ('Employees HeadCount', 57.142857142857146, 70)]
Filtered Matches for 'employees': ['SC11-17 employees', 'SC11-17 employees possessing a certification']
Potential Matches 

['SC11-17 employees possessing a certification',
 'Permanent CPH Female Employees',
 'Permanent Female Employees',
 'Female Representation',
 'SC11-17 employees',
 'Female CPH']

In [ ]:
get_top_matching_kpis("Which division in CAD has the highest % of Test Phishing Email Failure?")

Extracted Entities (Final Version): ['division', 'CAD', 'highest', '%', 'Test', 'Phishing', 'Email', 'Failure', 'division CAD', 'CAD highest', 'highest %', '% Test', 'Test Phishing', 'Phishing Email', 'Email Failure', 'division CAD highest', 'CAD highest %', 'highest % Test', '% Test Phishing', 'Test Phishing Email', 'Phishing Email Failure']
Potential Matches for 'division': [('Attrition', 47.05882352941177, 17), ('Hiring', 42.857142857142854, 79), ('Deviations Avoidance', 42.857142857142854, 257)]
Filtered Matches for 'division': []
Potential Matches for 'CAD': [('CDPNE', 50.0, 25), ('MVA', 33.33333333333333, 97), ('IKTVA', 25.0, 80)]
Filtered Matches for 'CAD': []
Potential Matches for '%': [('Certified Employee in Current GC', 0.0, 0), ('Manpower Tracked for Certification- Competency', 0.0, 1), ('Loss Time Injuries', 0.0, 2)]
Filtered Matches for '%': []
Potential Matches for 'Test': [('Number of Phishing Test Recipients', 100.0, 119), ('Test Phishing Email Failure', 100.0, 180), (

['Test Phishing Email Failure (Repeated Violators)',
 'Number of Failed Responses to the Phishing Tests',
 'Positive Responses to the Phishing Tests',
 'Number of Phishing Test Recipients',
 'Test Phishing Email Failure',
 'Test Phishing Email Neutral']

In [ ]:
get_top_matching_kpis("Which KPIs are underperforming for SSD in August 2024?")

Extracted Entities (Final Version): ['KPIs', 'SSD', 'August', 'KPIs SSD', 'SSD August', 'KPIs SSD August']
Potential Matches for 'KPIs': [('SMPs', 50.0, 172), ('Phishing KPI (Positive)', 29.629629629629633, 158), ('Transfer In', 26.66666666666667, 206)]
Filtered Matches for 'KPIs': []
Potential Matches for 'SSD': [('SMPs', 28.57142857142857, 172), ('SAP Systems', 28.57142857142857, 349), ('CDPNE', 25.0, 25)]
Filtered Matches for 'SSD': []
Potential Matches for 'August': [('Actual KMI Hours', 36.36363636363637, 242), ('SAP Systems', 35.294117647058826, 349), ('Actual KE Hours ()', 33.33333333333333, 241)]
Filtered Matches for 'August': []
Potential Matches for 'KPIs SSD': [('RDP In', 42.857142857142854, 161), ('SMPs', 33.33333333333333, 172), ('SAP Systems', 31.578947368421055, 349)]
Filtered Matches for 'KPIs SSD': []
Potential Matches for 'SSD August': [('RDP Out', 47.05882352941177, 162), ('On Budget DPSRs', 40.0, 132), ('Actual KMI Hours', 38.46153846153846, 242)]
Filtered Matches f

[]

In [ ]:
get_top_matching_kpis("How many patents were granted for SCMD in April 2024?")

Extracted Entities (Final Version): ['many', 'patents', 'SCMD', 'April', 'many patents', 'patents SCMD', 'SCMD April', 'many patents SCMD', 'patents SCMD April']
Filtered Matches for 'many': []
Filtered Matches for 'patents': ['Patents Filed', 'Patents Granted']
Filtered Matches for 'SCMD': []
Filtered Matches for 'April': []
Filtered Matches for 'many patents': []
Filtered Matches for 'patents SCMD': []
Filtered Matches for 'SCMD April': []
Filtered Matches for 'many patents SCMD': []
Filtered Matches for 'patents SCMD April': []


['Patents Granted', 'Patents Filed']

In [ ]:
get_top_matching_kpis("What was CAD Controllable Cost plan in the beginning of 2024?")

Extracted Entities (Final Version): ['CAD', 'Controllable', 'CAD Controllable']
Filtered Matches for 'CAD': []
Filtered Matches for 'Controllable': ['Controllable Cost', 'Controllable Cost ($)', 'Direct Controllable Cost']
Filtered Matches for 'CAD Controllable': ['Controllable Cost', 'Controllable Cost ($)', 'Direct Controllable Cost']


['Direct Controllable Cost', 'Controllable Cost ($)', 'Controllable Cost']

In [ ]:
get_top_matching_kpis("What is the Professional Certification % in F&AD in July 2024?")

Extracted Entities (Final Version): ['Professional', 'Certification', '%', 'F&AD', 'July', 'Professional Certification', 'Certification %', '% F&AD', 'F&AD July', 'Professional Certification %', 'Certification % F&AD', '% F&AD July']
Filtered Matches for 'Professional': ['Professional Certification']
Filtered Matches for 'Certification': ['Professional Certification']
Filtered Matches for '%': []
Filtered Matches for 'F&AD': []
Filtered Matches for 'July': []
Filtered Matches for 'Professional Certification': ['Professional Certification']
Filtered Matches for 'Certification %': ['Professional Certification']
Filtered Matches for '% F&AD': []
Filtered Matches for 'F&AD July': []
Filtered Matches for 'Professional Certification %': ['Professional Certification']
Filtered Matches for 'Certification % F&AD': []
Filtered Matches for '% F&AD July': []


['Professional Certification']

In [ ]:
get_top_matching_kpis("Which group in CAD has the highest % of female representation?")

Extracted Entities (Final Version): ['group', 'CAD', 'highest', '%', 'female', 'representation', 'group CAD', 'CAD highest', 'highest %', '% female', 'female representation', 'group CAD highest', 'CAD highest %', 'highest % female', '% female representation']
Filtered Matches for 'CAD': []
Filtered Matches for '%': []
Filtered Matches for 'female': ['Female CPH', 'Female Representation', 'Permanent Female Employees', 'Permanent CPH Female Employees']
Filtered Matches for 'representation': ['Female Representation']
Filtered Matches for 'group CAD': []
Filtered Matches for 'CAD highest': []
Filtered Matches for 'highest %': []
Filtered Matches for '% female': []
Filtered Matches for 'female representation': ['Female Representation']
Filtered Matches for 'group CAD highest': []
Filtered Matches for 'CAD highest %': []
Filtered Matches for 'highest % female': []
Filtered Matches for '% female representation': ['Female Representation']


['Permanent CPH Female Employees',
 'Permanent Female Employees',
 'Female Representation',
 'Female CPH']

In [ ]:
get_top_matching_kpis("Which division in CAD has the highest % of Test Phishing Email Failure?")

Extracted Entities (Final Version): ['division', 'CAD', 'highest', '%', 'Test', 'Phishing', 'Email', 'Failure', 'division CAD', 'CAD highest', 'highest %', '% Test', 'Test Phishing', 'Phishing Email', 'Email Failure', 'division CAD highest', 'CAD highest %', 'highest % Test', '% Test Phishing', 'Test Phishing Email', 'Phishing Email Failure']
Filtered Matches for 'division': []
Filtered Matches for 'CAD': []
Filtered Matches for '%': []
Filtered Matches for 'Test': ['Number of Phishing Test Recipients', 'Test Phishing Email Failure', 'Test Phishing Email Failure (Repeated Violators)']
Filtered Matches for 'Phishing': ['Positive Responses to the Phishing Tests', 'Number of Failed Responses to the Phishing Tests', 'Number of Phishing Test Recipients']
Filtered Matches for 'Email': ['Test Phishing Email Failure', 'Test Phishing Email Failure (Repeated Violators)', 'Test Phishing Email Neutral']
Filtered Matches for 'Failure': ['Test Phishing Email Failure', 'Test Phishing Email Failure (R

['Test Phishing Email Failure (Repeated Violators)',
 'Number of Failed Responses to the Phishing Tests',
 'Positive Responses to the Phishing Tests',
 'Number of Phishing Test Recipients',
 'Test Phishing Email Failure',
 'Test Phishing Email Neutral']